# Preprocessing of annotated data

In [ ]:
import os
import numpy
import cv2
import shutil as sh
import matplotlib.pyplot as plt

In [ ]:
pwd

**Setting up directories**

In [ ]:
# Select path to directories of annotated data
img_orig = r'C:\Users\grofc\Desktop\data_annotated\Original'
img_unmatched = r'C:\Users\grofc\Desktop\data_annotated\Unmatched'
img_matched = r'C:\Users\grofc\Desktop\data_annotated'

In [ ]:
#Function loads all files to proprietary directory
#Param: dir_path - path to proprietary directory

def load_dir_files(dir_path):
    files = None
    
    for file in os.listdir(dir_path):
        if dir_path:
            files = [os.path.join(dir_path,f) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(os.path.abspath(dir_path), f))]
        return files

In [ ]:
# Load the image names to lists
unmatched = load_dir_files(img_unmatched)
original = load_dir_files(img_orig)
matched = load_dir_files(img_matched)

In [ ]:
# Check their length
print(len(original))
print(len(unmatched))
print(len(matched))

**Preview Example**

In [ ]:
a = cv2.imread(matched[0], cv2.IMREAD_UNCHANGED)
b = cv2.imread(original[0], cv2.IMREAD_UNCHANGED)

In [ ]:
plt.imshow(a)

In [ ]:
plt.imshow(a, cmap='gray')

In [ ]:
plt.imshow(cv2.cvtColor(b, cv2.COLOR_BGR2RGB))

**Image Cropping**

In [ ]:
# Function crops image to 288 x 288 pixel size for Max pooling layers (Within excluded borders)
# Param: img - img to be cropped

def crop(img):
    #Croping grayscale images (masks)
    if len(img.shape) == 2:
        if img.shape[0] == 299 and img.shape[1] == 299:

            img = img[6:294, 6:294]
            return img
    else:
        #RGB original images
        if img.shape[0] == 299 and img.shape[1] == 299:
            img = img[6:294, 6:294, :]
            return img
        

**Image Extraction**

In [ ]:
# Function extracts source images and mask to their corresponding folder (E.g 'Lysto_pilot_1/(img|mask)(image.png|mask.png)')
# Param: destination - destination directory path for extracted images
# Param: original - directory path for original images
# Param: annotated - directory path for annotated images (masks)

def copy_files(destination, original, annotated):

    orig_dir = r'\img'  #Directory suffix for original img
    mask_dir = r'\mask' #Directory suffix for mask 
    
    orig = []
    mask = []
    
    orig_names = [''.join(x.split('\\')[-1].split('.png')) for x in original]
    orig_path = '\\'.join(''.join(original[0].split('lysto')[0]).split('\\')[:-1])
    
    #Parameters check
    if not isinstance(annotated, list):
        print(f'Invalid annotated img directory: {annotated}')
        return
    elif not isinstance(original, list):
        print(f'Invalid original img directory: {original}')
        return
    
    #Make destination directory if does not exist
    if not os.path.exists(destination):
        os.makedirs(destination)
        
    #Iterate over masks names
    for file in annotated:
        name = ''.join(file.split('\\')[-1].split('_mask')[0])
        orig_name = orig_path + '\\' + name + '.png'
        #print(name, orig_name)
        
        #Check if mask has specific original image and add them to lists
        if name in orig_names:
            mask.append(file)
            orig.append(orig_name)
    
    #Iterate over mask and original images (default sorted alphabetically)
    for f,o in zip(mask, orig):
        name = ''.join(f.split('\\')[-1].split('_mask')[0])

        #Image loading
        i_m = cv2.imread(f,-1)
        i_o = cv2.imread(o, -1)
        
        #Image cropping
        i_m = crop(i_m)
        i_o = crop(i_o)

        if i_m is None or  i_o is None:
            continue
        else:

            #Saving image to disk
            cv2.imwrite(f, i_m)
            cv2.imwrite(o, i_o)

            dst = destination + '\\' + name
            src = destination + '\\' + name +  '\\' + orig_dir
            mask_src = destination + '\\' + name + '\\' + mask_dir
            
            #Make directory structure and copy them there
            os.makedirs(dst)
            os.makedirs(src)
            os.makedirs(mask_src)
            sh.copy2(f, mask_src  + "\\" + name + '_mask.png')
            sh.copy2(o, src + "\\" + name + '.png')

In [ ]:
# Run to preprocess data
# Please be patient, it can take some time to finish....
copy_files(r'C:\\Users\\grofc\\Desktop\\Preprocessed', original, matched)